In [2]:
from multiprocessing import Pool
import os
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from cnmaps import draw_maps,get_adm_maps
import cmaps
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from multiprocessing import Process
p_cri = 0.01
def cut_edge(data):
    if len(data.shape) == 3:
        data = data[:, 7:-7, 7:-7]
    if len(data.shape) == 2:
        data = data[7:-7, 7:-7]
    return data
    
def maskout(data,mask):
    #data is 3d, mask is 2d
    mask_useid=np.where(mask>0,1,np.nan)
    mask_useid=np.expand_dims(mask_useid,axis=0)
    mask_useid=np.repeat(mask_useid,data.shape[0],axis=0)
    data_maskout=data*mask_useid
    data_maskout[:,:,0:70]=np.nan
    return data_maskout

def mask2dout(data,mask):
    #data is 2d, mask is 2d
    mask_useid=np.where(mask>0,1,np.nan)
    data_maskout=data*mask_useid
    data_maskout[:,0:70]=np.nan
    return data_maskout

CNmask = np.array(Dataset("CN_Subregion_new.nc")["reg_mask"])
CNmask = cut_edge(CNmask)

In [ ]:
event_types = ["spi_dry", "sti_hot", "chd_chd"]
models = ['ERA5','CESM','MPI','CESM_CWRF','MPI_CWRF']
trend_dict_fil = Dataset(f"checkcheck.nc")
freq_dict = {}
event = "chd_chd"
for model in models:
    for pid in ['hist','ssp585','ssp245']:
        if model == "ERA5" and pid in ["ssp585", "ssp245"]:
            continue
        freq_dict[f"{model}_{pid}_{event}_freq"] = maskout(trend_dict_fil[f"{model}_{pid}_{event}_freq"][:],CNmask)
freq_dict.keys()
freq_dict['CESM_hist_chd_chd_freq'].shape
merged_dict = {}
for model in ['CESM','MPI','CESM_CWRF','MPI_CWRF']:
    for pid in ['ssp585','ssp245']:
        hist = freq_dict[f"{model}_hist_{event}_freq"]
        fut = freq_dict[f"{model}_{pid}_{event}_freq"]
        merged_dict[f"{model}_{pid}_{event}_MERGED"] = np.concatenate([hist, fut], axis=0)
print(merged_dict.keys())
print(merged_dict['CESM_ssp585_chd_chd_MERGED'].shape)
ref_mat = merged_dict['CESM_ssp585_chd_chd_MERGED']
slope_dict = {}
intercept_dict = {}
mean_dict = {}
for model in ['CESM','MPI','CESM_CWRF','MPI_CWRF']:
    for pid in ['ssp585','ssp245']:
        mat_local = merged_dict[f"{model}_{pid}_{event}_MERGED"]
        line = np.nanmean(mat_local, axis=(1,2))
        x = np.arange(line.shape[0])
        y = line
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
        slope_dict[f"{model}_{pid}_{event}"] = slope
        intercept_dict[f"{model}_{pid}_{event}"] = intercept
        mean_dict[f"{model}_{pid}_{event}"] = line

In [ ]:
palette = ['green', 'red', 'blue', 'pink', 'lightblue']
models = ['ERA5','CESM','MPI','CESM_CWRF','MPI_CWRF']
def draw_sub_legend(axin):
    axinins = axin.inset_axes([0.025, 0.7, 0.45, 0.2])
    for mid, model in enumerate(models[:]):
        axinins.plot(-0.07,5-mid, color=palette[mid], mew=1.4,mec='grey', marker='d', markersize=8)
        axinins.plot(0,5-mid, color=palette[mid], marker='o', markersize=5)
        axinins.plot([0.05,0.2],[5-mid,5-mid], lw=2.5, color=palette[mid], markersize=5)
        # axinins.text(0.85, 5-mid, model, fontsize=15, va='center', ha='right')
        axinins.text(0.25, 5-mid, model, fontweight='bold', fontsize=15, va='center', ha='left')
    axinins.set_xlim(-0.12, 1.2)
    axinins.set_ylim(1,6)
    axinins.set_xticks([])
    axinins.set_yticks([])
    #set axis color
    axinins.spines['bottom'].set_color('gray')
    axinins.spines['top'].set_color('gray')
    axinins.spines['right'].set_color('gray')
    axinins.spines['left'].set_color('gray')

event = 'chd_chd'
for scenario in ['ssp585','ssp245']:
    fig = plt.figure(figsize=(6, 8), dpi=300)
    ax = fig.add_subplot(111)
    models = ['CESM','CESM_CWRF','MPI','MPI_CWRF']
    palette = ['pink', 'red', 'lightblue', 'blue']
    markers = ['d', 'd', 'd', 'd']
    for mid, model in enumerate(models):
        means = mean_dict[f"{model}_{scenario}_{event}"]
        trend = slope_dict[f"{model}_{scenario}_{event}"]
        intercept = intercept_dict[f"{model}_{scenario}_{event}"]
        x = np.arange(90)
        ymean = means
        y = trend*x + intercept
        x2 = (0.2 - intercept) / trend
        x3 = (0.3 - intercept) / trend
        x4 = (0.4 - intercept) / trend
        x5 = (0.5 - intercept) / trend
        ax.scatter(x, ymean, color=palette[mid],s=8)
        ax.plot(x, y,
                    color=palette[models.index(model)],
                    zorder=100,
                    lw=1.5,
                    label=model)
        ax.plot( x2,0.2, markers[mid], mew=1.4,mec='grey',color=palette[mid], markersize=10,zorder=101)
        ax.plot( x3,0.3, markers[mid], mew=1.4,mec='grey',color=palette[mid], markersize=10,zorder=101)
        ax.plot( x4,0.4, markers[mid], mew=1.4,mec='grey',color=palette[mid], markersize=10,zorder=101)
        ax.plot( x5,0.5, markers[mid], mew=1.4,mec='grey',color=palette[mid], markersize=10,zorder=101)
    ax.text(0.02,0.98,"a)",transform=ax.transAxes,fontsize=24,fontweight='bold',va='top',ha='left')
    ax.axhline(0.5, color='black', lw=0.5, ls='--', zorder=101)
    ax.axhline(0.4, color='black', lw=1.0, ls='--', zorder=101)
    ax.axhline(0.3, color='black', lw=1.5, ls='--', zorder=101)
    ax.axhline(0.2, color='black', lw=2.0, ls='--', zorder=101)
    ax.text(0.96,0.02,scenario.upper(),transform=ax.transAxes,fontsize=15,va='bottom',ha='right',fontweight='bold')
    ax.legend().remove()
    ax.axvspan(45, max(x)+2, color='gray', alpha=0.3)
    ax.set_ylim(0, 0.75)
    ax.set_xticks(np.arange(0, 91, 10))
    ax.set_xticklabels(np.arange(1970, 2061, 10), fontsize=10)
    ax.set_yticks(np.arange(0, 0.8, 0.1))
    ax.set_yticklabels(['0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7'], fontsize=15)
    ax.set_xlim(-1, 90)
    ax.set_xlabel('Year', fontsize=15)
    draw_sub_legend(ax)
    eventname='CDHEs'
    ax.set_ylabel('Regional mean '+eventname+" frequency", fontsize=15)
    plt.tight_layout()
    plt.savefig(f"trend_{scenario}_{event}.png")



In [29]:
def dealneg(trend,intercept):
    trendpositive = np.where(trend>0.005,trend,np.nan)
    interceptpositive = np.where(trend>0.005,intercept,np.nan)
    trend = np.where(trend>0.005,trend,np.nanmean(trendpositive))
    intercept = np.where(trend>0.005,intercept,np.nanmean(interceptpositive))
    return trend,intercept

def first_time_hit(data, trend, intercept, hit):
    hit_time = np.ndarray((data.shape[1], data.shape[2]))
    trend, intercept = dealneg(trend, intercept)
    for ilat in range(data.shape[1]):
        for ilon in range(data.shape[2]):
            hit_time[ilat, ilon] = (hit - intercept[ilat, ilon]) / (trend[ilat, ilon]/10) + 2015
    hit_time = mask2dout(hit_time, CNmask)
    hit_time = np.where(hit_time > 10000, np.nan, hit_time)
    return hit_time

In [30]:
CNregionfil=Dataset("./CN_Subregion_new.nc")
lat2d=np.array(CNregionfil.variables["latitude"][:])
lon2d=np.array(CNregionfil.variables["longitude"][:])
lat2d=cut_edge(lat2d)
lon2d=cut_edge(lon2d)
event_types = ["spi_dry", "sti_hot", "chd_chd"]
models = ['ERA5','CESM','MPI','CESM_CWRF','MPI_CWRF']
trend_dict_fil = Dataset(f"checkcheck.nc")

df = pd.DataFrame(columns=['year','model','value'])
df_mean = pd.DataFrame(columns=['year','model','period','mean'])
df_regress = pd.DataFrame(columns=['model','period','slope','intercept'])
event = "chd_chd"
event_first_time = {}
for model in models:
    for pid in ['hist','ssp585','ssp245']:
        if model == "ERA5" and pid in ["ssp585", "ssp245"]:
            continue
        data = np.array(trend_dict_fil[f"{model}_{pid}_{event}_freq"][:])
        data = maskout(data, CNmask)
        trend = mask2dout(np.array(trend_dict_fil[f"{model}_{pid}_{event}_freq_trend"][:]), CNmask)
        intercept = mask2dout(np.array(trend_dict_fil[f"{model}_{pid}_{event}_freq_intercept"][:]), CNmask)
        for threshold in [0.3,0.4,0.5,0.6,0.7]:
            event_first_time[f"{model}_{pid}_{event}_{threshold}"] = first_time_hit(data,trend,intercept,threshold)


In [31]:
CNregionfil=Dataset("./newmask2.nc")
CNmaskraw=CNregionfil.variables["reg_mask"][:]
CNmaskraw=np.where(CNmaskraw==7,-1,CNmaskraw)
CNmaskraw=np.where(CNmaskraw==8,-1,CNmaskraw)
CNmaskraw=np.where(CNmaskraw==9,-1,CNmaskraw)
CNmask=cut_edge(CNmaskraw)

In [32]:
def draw_mat(period,figname,lon,lat,matrix,modelname,varname,diff_levs,cm,CNmaskraw,figl):
    thresh=figname[-1]
    matrix=mask2dout(matrix,CNmask)
    fig=plt.figure(figsize=(5,4))
    proj = ccrs.PlateCarree()  # 创建坐标系
    cwrf_cnproj = ccrs.LambertConformal(central_longitude=110.0, central_latitude=35.17781, false_easting=0.0, false_northing=0.0,  standard_parallels = (30, 60), globe=None, cutoff=-30)
    ax = plt.subplot(1, 1,1, projection=cwrf_cnproj)
    ax.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.3, zorder=99)
    ax.add_feature(cfeat.OCEAN, facecolor="white", zorder=10)
    draw_maps(get_adm_maps(level='国'),color='grey',zorder=99,linewidth=0.8)
    draw_maps(get_adm_maps(level='省'),color='grey',zorder=99,linewidth=0.4)
    ax.set_extent([100,133,17.2,54])
    cn=ax.pcolormesh(lon,lat,matrix,cmap=cm,vmin=diff_levs[0],vmax=diff_levs[-1],transform = ccrs.PlateCarree())
    # cn=ax.contourf(lon,lat,matrix,levels=diff_levs,cmap=cm,extend='both',transform = ccrs.PlateCarree())
    lon1d=lon.flatten()
    lat1d=lat.flatten()
    CNmaskraw_local=CNmaskraw
    CNmaskraw_local=np.where(CNmaskraw_local==-1,-1,np.nan)
    ax.contourf(lon,lat,CNmaskraw_local,levels=[-1,0],colors='lightgrey',transform = ccrs.PlateCarree(),zorder=199)
    ax.text(0.01,0.98,figl+") "+modelname,transform=ax.transAxes,fontsize=14,ha='left',va='top',fontweight="bold")
    if modelname == "CESM":
        ax.text(-0.02,0.5,"Threshold = 0."+thresh,transform=ax.transAxes,fontsize=14,ha='right',va='center',fontweight="bold",rotation=90)
    savename = './hits/'+period+"_"+figname+"_"+modelname+"_"+varname+'.png'
    plt.savefig(savename,dpi=300,bbox_inches='tight')
    os.system("convert -trim "+savename+" "+savename)
    if modelname == "MPI_CWRF":
        fig=plt.figure(figsize=(2,15))
        cbar_ax = fig.add_axes([0.85, 0.07, 0.1, 0.9])
        cbar=plt.colorbar(cn,cax=cbar_ax,label="",orientation='vertical',shrink=0.02,extend='both')
        cbar.set_ticks(diff_levs[::1])
        cbar.ax.tick_params(labelsize=22)
        name="CDHE"
        cbar.set_label("Year when regressed frequency reaches threshold",fontsize=24)
        plt.savefig("./hits/CHDcolorbar_"+thresh+".png",bbox_inches='tight')


In [ ]:

lets3 = ['b','c','d','e']
lets4 = ['f','g','h','i']
lets5 = ['j','k','l','m']
lets6 = ['x','x','x','x']

levs5 = np.arange(2015, 2061, 5)
levs3 = levs5
levs4 = levs5
levs6 = levs5
cmapuse = cmaps.WhiteYellowOrangeRed_r
for ithresh, thresh in enumerate([0.6,0.3,0.4,0.5]):
    for imodel, model in enumerate(['CESM','CESM_CWRF','MPI','MPI_CWRF']):
        for iperiod, period in enumerate(['hist','ssp245','ssp585']):
            levs=levs5
            lets=eval(f"lets{str(thresh).split('.')[-1]}")
            job = Process(target=draw_mat, args=(period,'first_time'+str(thresh)[-1],lon2d,lat2d,event_first_time[f"{model}_{period}_{event}_{thresh}"],model,event,levs,cmapuse,CNmask,lets[imodel]))
            job.start()
job.join()



In [35]:
!convert +append hits/ssp585_first_time3_CESM_chd_chd.png hits/ssp585_first_time3_CESM_CWRF_chd_chd.png hits/ssp585_first_time3_MPI_chd_chd.png hits/ssp585_first_time3_MPI_CWRF_chd_chd.png  hits/ssp585_first_time3.png
!convert +append hits/ssp585_first_time4_CESM_chd_chd.png hits/ssp585_first_time4_CESM_CWRF_chd_chd.png hits/ssp585_first_time4_MPI_chd_chd.png hits/ssp585_first_time4_MPI_CWRF_chd_chd.png hits/ssp585_first_time4.png
!convert +append hits/ssp585_first_time5_CESM_chd_chd.png hits/ssp585_first_time5_CESM_CWRF_chd_chd.png hits/ssp585_first_time5_MPI_chd_chd.png hits/ssp585_first_time5_MPI_CWRF_chd_chd.png  hits/ssp585_first_time5.png
!convert -append hits/ssp585_first_time3.png hits/ssp585_first_time4.png hits/ssp585_first_time5.png hits/ssp585_first_time35.png
!convert -resize x2787 hits/CHDcolorbar_5.png hits/ssp585_CHDcolorbar_5.png
!convert +append hits/ssp585_first_time35.png hits/ssp585_CHDcolorbar_5.png hits/ssp585_first_time.png

!convert +append hits/ssp245_first_time3_CESM_chd_chd.png hits/ssp245_first_time3_CESM_CWRF_chd_chd.png hits/ssp245_first_time3_MPI_chd_chd.png hits/ssp245_first_time3_MPI_CWRF_chd_chd.png  hits/ssp245_first_time3.png
!convert +append hits/ssp245_first_time4_CESM_chd_chd.png hits/ssp245_first_time4_CESM_CWRF_chd_chd.png hits/ssp245_first_time4_MPI_chd_chd.png hits/ssp245_first_time4_MPI_CWRF_chd_chd.png hits/ssp245_first_time4.png
!convert +append hits/ssp245_first_time5_CESM_chd_chd.png hits/ssp245_first_time5_CESM_CWRF_chd_chd.png hits/ssp245_first_time5_MPI_chd_chd.png hits/ssp245_first_time5_MPI_CWRF_chd_chd.png  hits/ssp245_first_time5.png
!convert -append hits/ssp245_first_time3.png hits/ssp245_first_time4.png hits/ssp245_first_time5.png hits/ssp245_first_time35.png
!convert -resize x2787 hits/CHDcolorbar_5.png hits/ssp245_CHDcolorbar_5.png
!convert +append hits/ssp245_first_time35.png hits/ssp245_CHDcolorbar_5.png hits/ssp245_first_time.png



In [10]:
!convert -resize x2400 ./hits/ssp245_first_time.png ./hits/ssp245_first_time.png
!convert -resize x2400 ./hits/ssp585_first_time.png ./hits/ssp585_first_time.png

!convert +append ./trend_ssp245_chd_chd.png ./hits/ssp245_first_time.png ./ssp245_all.png
!convert +append ./trend_ssp585_chd_chd.png ./hits/ssp585_first_time.png ./ssp585_all.png